In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

from sklearn.model_selection import GridSearchCV


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Тут вибераемо усі файли в нашому датасеті

In [105]:
# Список класів активностей
classes = ["idle", "running", "stairs", "walking"]

# Порожній DataFrame для об'єднання всіх даних
all_data = pd.DataFrame()


# Проходимо по кожному класу
for activity in classes:
    # Визначаемо шлях до папки з відповідним класом на Google Drive
    folder_path = os.path.join("/content/drive/My Drive/data", activity)

    # Перевірка наявності папки перед завантаженням
    if os.path.exists(folder_path):
        # Проходимо по всім файлам в папці
        for filename in os.listdir(folder_path):
            if filename.endswith(".csv"):
                # Зчитуємо дані з CSV файлу
                file_path = os.path.join(folder_path, filename)
                data = pd.read_csv(file_path)

                # Додаемо стовпець activity з ім'ям класу
                data['activity'] = activity

                # Об'єднуемо дані з іншими даними
                all_data = pd.concat([all_data, data], ignore_index=True)
    else:
        print(f"Папка {folder_path} для класу {activity} не існує.")

In [106]:
# Розділення даних на навчальний та тестовий набори
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42)
print(train_data)
print("-----------")
print(test_data)


        accelerometer_X  accelerometer_Y  accelerometer_Z activity
43768         10.975020        12.962208        -3.849876  running
127052        -3.897760         7.541735        -2.738967  running
182122        -1.833958        -0.469264         2.801216  walking
79126         -1.221043         3.787627        -3.787627  running
148422        -1.853112        -2.389413         1.848324  walking
...                 ...              ...              ...      ...
119879         4.539407        -5.377377        -1.115698  running
103694         9.155427        28.151024        -0.861913  running
131932        -8.006210         5.315128         2.284068  running
146867        -2.255338        -5.573701        -8.049306  walking
121958         0.167594       -16.788143         0.675165  running

[155328 rows x 4 columns]
-----------
        accelerometer_X  accelerometer_Y  accelerometer_Z activity
96885          4.817134        15.169662        -0.445322  running
179740         2.039860

In [107]:
# Розрахунок часових ознак
time_features = ['mean', 'std', 'median']

features = []
for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']:
    for stat in time_features:
        feature_name = f"{axis}_{stat}"
        features.append(feature_name)
        train_data[feature_name] = train_data.groupby('activity')[axis].transform(stat)
        test_data[feature_name] = test_data.groupby('activity')[axis].transform(stat)
print(features)

['accelerometer_X_mean', 'accelerometer_X_std', 'accelerometer_X_median', 'accelerometer_Y_mean', 'accelerometer_Y_std', 'accelerometer_Y_median', 'accelerometer_Z_mean', 'accelerometer_Z_std', 'accelerometer_Z_median']


In [108]:
# Вибір фіч для навчання та тестування
X_train = train_data[features]
y_train = train_data['activity']
X_test = test_data[features]
y_test = test_data['activity']

In [109]:
# Навчання моделі SVM
svm_model = SVC()
svm_model.fit(X_train, y_train)

SVC()

In [110]:
# Прогноз на тестовому наборі та оцінка моделі SVM
svm_predictions = svm_model.predict(X_test)
print(svm_predictions)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("Точність моделі SVM:", svm_accuracy)

['running' 'walking' 'running' ... 'walking' 'stairs' 'stairs']
Точність моделі SVM: 1.0


In [111]:
# Навчання моделі Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)


RandomForestClassifier()

In [112]:
# Прогноз на тестовому наборі та оцінка моделі Random Forest
rf_predictions = rf_model.predict(X_test)
print(rf_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Точність моделі Random Forest:", rf_accuracy)

['running' 'walking' 'running' ... 'walking' 'stairs' 'stairs']
Точність моделі Random Forest: 1.0


In [113]:
# Навчання моделі SVM з підтримкою ймовірностей
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)

SVC(probability=True)

In [114]:
# Прогноз ймовірностей на тестовому наборі
svm_probabilities = svm_model.predict_proba(X_test)
print(svm_probabilities)

[[5.28347363e-10 9.99986455e-01 6.40901311e-06 7.13512462e-06]
 [1.65400046e-09 1.24913706e-09 1.86154750e-09 9.99999995e-01]
 [5.28347363e-10 9.99986455e-01 6.40901311e-06 7.13512462e-06]
 ...
 [1.65400046e-09 1.24913706e-09 1.86154750e-09 9.99999995e-01]
 [1.37292071e-07 1.02238775e-07 9.99928092e-01 7.16680661e-05]
 [1.37292071e-07 1.02238775e-07 9.99928092e-01 7.16680661e-05]]


In [115]:
# Обчислення ROC AUC для моделі SVM
svm_roc_auc = roc_auc_score(y_test, svm_probabilities, multi_class='ovr')
print(f"ROC AUC for SVM: {svm_roc_auc}")

ROC AUC for SVM: 1.0


In [116]:
# Прогноз на тестовому наборі та оцінка моделі SVM
svm_predictions = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("Точність моделі SVM:", svm_accuracy)

Точність моделі SVM: 1.0


In [117]:
# Навчання моделі Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [125]:
# Прогноз на тестовому наборі та оцінка моделі Random Forest
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Точність моделі Random Forest:", rf_accuracy)

Точність моделі Random Forest: 1.0


In [126]:
# Порівняння результатів моделей

# Модель SVM
print("Модель SVM:")
svm_confusion = confusion_matrix(y_test, svm_predictions)
print("Матриця плутанини:")
print(svm_confusion)

Модель SVM:
Матриця плутанини:
[[ 6209     0     0     0]
 [    0 20448     0     0]
 [    0     0   964     0]
 [    0     0     0 11211]]


In [120]:
print("\nЗвіт про класифікації:")
svm_classification_report = classification_report(y_test, svm_predictions)
print(svm_classification_report)


Звіт про класифікації:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6209
     running       1.00      1.00      1.00     20448
      stairs       1.00      1.00      1.00       964
     walking       1.00      1.00      1.00     11211

    accuracy                           1.00     38832
   macro avg       1.00      1.00      1.00     38832
weighted avg       1.00      1.00      1.00     38832



In [121]:
# Модель Random Forest
print("\nМодель Random Forest:")
rf_confusion = confusion_matrix(y_test, rf_predictions)
print("Матриця плутанини:")
print(rf_confusion)
print("\nЗвіт про класифікації:")
rf_classification_report = classification_report(y_test, rf_predictions)
print(rf_classification_report)

rf_roc_auc = roc_auc_score(y_test, rf_model.predict_proba(X_test), multi_class='ovr')
print(f"ROC AUC: {rf_roc_auc}")


Модель Random Forest:
Матриця плутанини:
[[ 6209     0     0     0]
 [    0 20448     0     0]
 [    0     0   964     0]
 [    0     0     0 11211]]

Звіт про класифікації:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6209
     running       1.00      1.00      1.00     20448
      stairs       1.00      1.00      1.00       964
     walking       1.00      1.00      1.00     11211

    accuracy                           1.00     38832
   macro avg       1.00      1.00      1.00     38832
weighted avg       1.00      1.00      1.00     38832

ROC AUC: 1.0


In [122]:
# Тюнінг гіперпараметрів для SVM
param_grid_svm = {'C': [0.1, 1, 10],
                  'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
svm_grid = GridSearchCV(SVC(), param_grid_svm, cv=3)
svm_grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']})

In [123]:
# Найкращі гіперпараметри для SVM
best_params_svm = svm_grid.best_params_
print("\nНайкращі гіперпараметри для SVM:")
print(best_params_svm)


Найкращі гіперпараметри для SVM:
{'C': 0.1, 'kernel': 'linear'}


In [124]:
# Тюнінг гіперпараметрів для Random Forest
param_grid_rf = {'n_estimators': [50, 100, 200],
                 'max_depth': [None, 10, 20, 30]}
rf_grid = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=3)
rf_grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'n_estimators': [50, 100, 200]})

In [54]:
# Найкращі гіперпараметри для Random Forest
best_params_rf = rf_grid.best_params_
print("\nНайкращі гіперпараметри для Random Forest:")
print(best_params_rf)


Найкращі гіперпараметри для Random Forest:
{'max_depth': None, 'n_estimators': 50}


In [127]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Метрика точності (Precision)
precision = precision_score(y_test, svm_predictions, average='weighted')
print("Точність (Precision) моделі SVM:", precision)

# Метрика відгук (Recall)
recall = recall_score(y_test, svm_predictions, average='weighted')
print("Відгук (Recall) моделі SVM:", recall)

# F1-мера
f1 = f1_score(y_test, svm_predictions, average='weighted')
print("F1-мера моделі SVM:", f1)

Точність (Precision) моделі SVM: 1.0
Відгук (Recall) моделі SVM: 1.0
F1-мера моделі SVM: 1.0


In [56]:
# Метрика точності (Precision)
precision_rf = precision_score(y_test, rf_predictions, average='weighted')
print("Точність (Precision) моделі Random Forest:", precision_rf)

# Метрика відгук (Recall)
recall_rf = recall_score(y_test, rf_predictions, average='weighted')
print("Відгук (Recall) моделі Random Forest:", recall_rf)

# F1-мера
f1_rf = f1_score(y_test, rf_predictions, average='weighted')
print("F1-мера моделі Random Forest:", f1_rf)


Точність (Precision) моделі Random Forest: 1.0
Відгук (Recall) моделі Random Forest: 1.0
F1-мера моделі Random Forest: 1.0


In [128]:
# Матриця плутанини для моделі SVM
svm_confusion = confusion_matrix(y_test, svm_predictions)
print("Матриця плутанини для моделі SVM:")
print(svm_confusion)

# Матриця плутанини для моделі Random Forest
rf_confusion = confusion_matrix(y_test, rf_predictions)
print("Матриця плутанини для моделі Random Forest:")
print(rf_confusion)


Матриця плутанини для моделі SVM:
[[ 6209     0     0     0]
 [    0 20448     0     0]
 [    0     0   964     0]
 [    0     0     0 11211]]
Матриця плутанини для моделі Random Forest:
[[ 6209     0     0     0]
 [    0 20448     0     0]
 [    0     0   964     0]
 [    0     0     0 11211]]


In [129]:
# Визначення важливості функцій
feature_importances = rf_model.feature_importances_
sorted_idx = feature_importances.argsort()[::-1]

# Виведення найважливіших функцій
for i, idx in enumerate(sorted_idx[:10]):
    print(f"{i+1}. {features[idx]}: {feature_importances[idx]}")


1. accelerometer_X_std: 0.16508872415657944
2. accelerometer_X_median: 0.1631131074905243
3. accelerometer_Y_mean: 0.15860299200285163
4. accelerometer_Y_std: 0.14032931686977998
5. accelerometer_Z_std: 0.13577603027383908
6. accelerometer_X_mean: 0.13180989257250722
7. accelerometer_Y_median: 0.095196147808294
8. accelerometer_Z_median: 0.005054540157271269
9. accelerometer_Z_mean: 0.0050292486683532
